# **Multiclass Classification**

Multiclass classification are those predictive modeling problems where examples are assigned one or more than two classes. The problem here is to predict numeric value from hydroponic parameters, where each class is assigned a unique integer value as the label from 0 to 17. The goal is to predict the probability of the example belonging to each known class.

## **Label and Action**



*   Label 0 : No action
*   Label 1 : Pump Water
* Label 2 : Light On
* Label 3 : Pump Nutrient
* Label 4 : pH Up
* Label 5 : pH Down
* Label 6 : Pump Water + Light On
* Label 7 : Pump Water + pH Up
* Label 8 : Pump Water + pH Down
* Label 9 : Pump Water + Light On + pH Up
* Label 10 : Pump Water + Light On + pH Down
* Label 11 : Light On + Pump Nutrient
* Label 12 : Light On + pH Up
* Label 13 : Light On + pH Down
* Label 14 : Light On + Pump Nutrient + pH Up
* Label 15 : Light On + Pump Nutrient + pH Down
* Label 16 : Pump Nutrient + pH Up
* Label 17 : Pump Nutrient + pH Down





# **Import Library**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten

print(tf.__version__)

2.8.2


## **Load Data**

In [ ]:
# import pandas as pd

# url = 'https://raw.githubusercontent.com/HydroMon/hydromon-machine-learning/main/dataset/ver2/merged_dataset.csv'
# col_lists = ["pH", "TDS", "Light Intensity", "Label"]
# df = pd.read_csv(url, usecols=col_lists)

# df

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer

html = requests.get('https://github.com/HydroMon/hydromon-machine-learning/tree/main/dataset/ver2')

dfs = []
for link in BeautifulSoup(html.text, parse_only=SoupStrainer('a')):
    if hasattr(link, 'href') and link['href'].endswith('.csv'):
        url = 'https://github.com'+ link['href'].replace('/blob/', '/raw/')
        dfs.append(pd.read_csv(url))
df = pd.concat(dfs)

df

,Date,Time,ID,TDS,Light Intensity,pH,Air Temperature,Humidity,EC,Label
0,2022-05-20,11:34:05,40TWL4,993,689,6.268,31,98,2.248,0
1,2022-05-20,11:34:10,40TWL4,735,462,6.044,27,46,2.887,0
2,2022-05-20,11:34:15,40TWL4,750,778,5.599,38,53,2.953,0
3,2022-05-20,11:34:20,40TWL4,934,491,6.090,38,45,2.114,0
4,2022-05-20,11:34:25,40TWL4,769,754,6.253,28,27,3.499,0
...,...,...,...,...,...,...,...,...,...,...
995,2022-05-20,01:27:00,40TWL4,1378,880,4.014,29,19,3.097,9
996,2022-05-20,01:27:05,40TWL4,1023,846,4.585,23,78,3.117,9
997,2022-05-20,01:27:10,40TWL4,1219,969,4.728,38,87,2.850,9
998,2022-05-20,01:27:15,40TWL4,1019,866,5.272,20,23,3.179,9


In [ ]:
# use certain features only
df = df.loc[:, ["TDS", "Light Intensity", "pH", "Label"]]
df 

,TDS,Light Intensity,pH,Label
0,993,689,6.268,0
1,735,462,6.044,0
2,750,778,5.599,0
3,934,491,6.090,0
4,769,754,6.253,0
...,...,...,...,...
995,1378,880,4.014,9
996,1023,846,4.585,9
997,1219,969,4.728,9
998,1019,866,5.272,9


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TDS              18000 non-null  int64  
 1   Light Intensity  18000 non-null  int64  
 2   pH               18000 non-null  float64
 3   Label            18000 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 703.1 KB


## **Data Preprocessing**

In [ ]:
# data shuffling
df = df.sample(frac=1).reset_index(drop=True)
df

,TDS,Light Intensity,pH,Label
0,1223,919,4.172,9
1,1014,755,7.615,8
2,918,703,4.928,4
3,574,420,6.171,3
4,784,868,5.057,12
...,...,...,...,...
17995,445,996,6.879,15
17996,1322,948,4.000,9
17997,963,432,4.504,4
17998,354,545,6.328,3


In [ ]:
# edit bagian sini
normalization = df['TDS'].max(),df['Light Intensity'].max(),df['pH'].max()
 
print("Maximum value in column 'x': " )
print(normalization)

Maximum value in column 'x': 
(1500, 1000, 8.0)


In [ ]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))

# scaled feature
df_after_min_max_scaler = min_max_scaler.fit_transform(df)
df_after_min_max_scaler

array([[7.20833333e-01, 7.88333333e-01, 6.35250000e-01, 5.88235294e-01],
       [8.33333333e-04, 6.35000000e-01, 1.49250000e-01, 9.41176471e-01],
       [3.96666667e-01, 1.50000000e-02, 7.01000000e-01, 2.94117647e-01],
       ...,
       [1.53333333e-01, 6.80000000e-01, 6.06250000e-01, 6.47058824e-01],
       [2.97500000e-01, 7.25000000e-01, 2.08500000e-01, 8.23529412e-01],
       [1.31666667e-01, 7.10000000e-01, 7.63250000e-01, 8.82352941e-01]])

In [ ]:
from sklearn import preprocessing

cols_to_norm = ['pH', 'TDS', 'Light Intensity']

# min max scaler
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))

#scaled feature
df_after_min_max_scaler = min_max_scaler.fit_transform(df[cols_to_norm])
# print(df_after_min_max_scaler.shape)

# convert to dataframe
column = ['pH','TDS','Light Intensity']
df_norm = pd.DataFrame(df_after_min_max_scaler, columns=column)
df_norm

,pH,TDS,Light Intensity
0,0.63525,0.720833,0.788333
1,0.14925,0.000833,0.635000
2,0.70100,0.396667,0.015000
3,0.58200,0.554167,0.545000
4,0.56475,0.980000,0.540000
...,...,...,...
17995,0.11850,0.911667,0.046667
17996,0.54950,0.265000,0.138333
17997,0.60625,0.153333,0.680000
17998,0.20850,0.297500,0.725000


In [ ]:
# feature scaling (normalize) to make data have similar distribution
# min-max feature scaling
cols_to_norm = ['pH','TDS','Light Intensity'] 
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df

,TDS,Light Intensity,pH,Label
0,0.418333,0.935000,0.43000,2
1,0.869167,0.765000,0.86775,10
2,0.242500,0.608333,0.57800,3
3,0.195000,0.498333,0.26275,16
4,0.170833,0.931667,0.42200,11
...,...,...,...,...
17995,0.463333,0.595000,0.74300,5
17996,0.301667,0.261667,0.50375,3
17997,0.654167,0.015000,0.87725,8
17998,0.138333,0.218333,0.23275,16


In [ ]:
# split into input and output columns
# input = pH, TDS, Light Intensity
# output = labels
X, y = df.values[:, :-1], df.values[:, -1]

In [ ]:
# input
print("Input Data:")
print(X)

# output
print("Output: ")
print(y)

Input Data:
[[0.36583333 0.23333333 0.33275   ]
 [0.27416667 0.55       0.0555    ]
 [0.94166667 0.27666667 0.401     ]
 ...
 [0.73       0.72       0.2055    ]
 [0.45166667 0.59166667 0.608     ]
 [0.28666667 0.99166667 0.14275   ]]
Output: 
[ 4. 16.  1. ...  9.  0. 14.]


In [ ]:
# ensure all input data are floating point values
X = X.astype('float32')
X

array([[0.36583334, 0.23333333, 0.33275   ],
       [0.27416667, 0.55      , 0.0555    ],
       [0.94166666, 0.27666667, 0.401     ],
       ...,
       [0.73      , 0.72      , 0.2055    ],
       [0.45166665, 0.59166664, 0.608     ],
       [0.28666666, 0.9916667 , 0.14275   ]], dtype=float32)

In [ ]:
# split into input and output columns
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(14400, 3) (3600, 3) (14400,) (3600,)


In [ ]:
# determine the number of input features
# input = pH, TDS, Light Intensity
n_features = X_train.shape[1]
n_features

3

## **Create Model**

The function requires that the output layer is configured with an n nodes (one for each class), in this case 18 nodes, and a ‘softmax‘ activation in order to predict the probability for each class.

In [ ]:
# define model
tf.random.set_seed(42)

model = Sequential()
model.add(Flatten(input_shape=(n_features,))) # input layer
# model.add(Dense(10, activation='relu', kernel_initializer='he_normal')) # hidden layer
# model.add(Dense(8, activation='relu', kernel_initializer='he_normal')) # hidden layer
model.add(Dense(64, activation='relu', kernel_initializer='he_normal')) # hidden layer
model.add(Dense(32, activation='relu', kernel_initializer='he_normal')) # hidden layer
model.add(Dense(16, activation='relu', kernel_initializer='he_normal')) # hidden layer
model.add(Dense(18, activation='softmax')) # output layer

In this section, we will investigate loss functions that are approriate for multiclass classification predictive modeling problems.

In [ ]:
# compile the model
# model trains using Adam optimizer with learning rate = 0.001
# you can increase learning rate to 0.1
# model calculates loss using sparse_categorical_crossentropy that is used to 
model.compile(optimizer=tf.optimizers.Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3)                 0         
                                                                 
 dense (Dense)               (None, 64)                256       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 18)                306       
                                                                 
Total params: 3,170
Trainable params: 3,170
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# fit the model
history = model.fit(
    x=X_train,  
    y=y_train, 
    epochs=100, # train for 100 epochs 
    batch_size=32) #use 32 examples per batch

# batch = 128

Epoch 1/100
450/450 [==============================] - 1s 2ms/step - loss: 1.7065 - accuracy: 0.4485
Epoch 2/100
450/450 [==============================] - 1s 2ms/step - loss: 0.6126 - accuracy: 0.7896
Epoch 3/100
450/450 [==============================] - 1s 3ms/step - loss: 0.3437 - accuracy: 0.8842
Epoch 4/100
450/450 [==============================] - 1s 2ms/step - loss: 0.2492 - accuracy: 0.9195
Epoch 5/100
450/450 [==============================] - 1s 2ms/step - loss: 0.2052 - accuracy: 0.9324
Epoch 6/100
450/450 [==============================] - 1s 2ms/step - loss: 0.1747 - accuracy: 0.9435
Epoch 7/100
450/450 [==============================] - 1s 2ms/step - loss: 0.1574 - accuracy: 0.9485
Epoch 8/100
450/450 [==============================] - 1s 2ms/step - loss: 0.1448 - accuracy: 0.9506
Epoch 9/100
450/450 [==============================] - 1s 2ms/step - loss: 0.1390 - accuracy: 0.9533
Epoch 10/100
450/450 [==============================] - 1s 2ms/step - loss: 0.1289 - accura

In [ ]:
# evaluate the model
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=2)
test_lost, test_acc = model.evaluate(X_test, y_test, verbose=2)
# loss, acc = model.evaluate(X_test, y_test, verbose=2)
print('Train accuracy: %.3f' % train_acc)
print('Test accuracy: %.3f' % test_acc)

450/450 - 1s - loss: 0.0557 - accuracy: 0.9766 - 579ms/epoch - 1ms/step
113/113 - 0s - loss: 0.0683 - accuracy: 0.9697 - 286ms/epoch - 3ms/step
Train accuracy: 0.977
Test accuracy: 0.970


##**Making Prediction**

In [ ]:
# make a prediction
# tds, light intensity, ph
from numpy import argmax
# row = [0.5775, 0.48166665, 0.567] # label 0
# row = [0.627500, 0.845000, 0.33150] # label 9
# row = [0.405000, 0.376667, 0.96125] # label 5
# row = [0.901667,	0.235000,	0.59900] # label 1
row = [0.550833,	0.716667,	0.44050] # label 2

yhat = model.predict([row], verbose=1)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

1/1 [==============================] - 0s 96ms/step
Predicted: [[7.5784115e-08 1.5792241e-10 9.9888200e-01 6.9058543e-20 8.3405720e-21
  6.3232157e-26 1.1178378e-03 2.1294567e-21 4.0060554e-26 8.3840036e-17
  5.0225616e-23 2.4863214e-14 1.1864473e-13 2.6120936e-20 3.9485376e-19
  6.2085843e-33 2.0813322e-38 0.0000000e+00]] (class=2)


In [ ]:
df.head()

,TDS,Light Intensity,pH,Label
0,0.835000,0.761667,0.62000,6
1,0.025000,0.160000,0.56400,3
2,0.680833,0.783333,0.57500,6
3,0.497500,0.000000,0.95175,5
4,0.451667,0.175000,0.22075,4


In [ ]:
0.6275 * 1500


941.2499999999999

In [ ]:
0.845000 * 1000

845.0

In [ ]:
0.33150 * 8

2.652

In [ ]:
# make a prediction with normalization
# tds, light intensity, ph
import numpy
from numpy import argmax
# row = [0.5775, 0.48166665, 0.567] # label 0
row = numpy.array([1000, 750, 6]) # label 9
# row = 
predict_row = row/normalization
predict_row1 = predict_row.tolist()
yhat = model.predict([predict_row1])
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

Predicted: [[1.0659668e-35 3.4292693e-27 1.2479373e-24 0.0000000e+00 0.0000000e+00
  5.5408926e-20 3.4814849e-16 0.0000000e+00 1.1937662e-10 0.0000000e+00
  1.0000000e+00 0.0000000e+00 0.0000000e+00 2.2150171e-10 0.0000000e+00
  2.6247926e-34 0.0000000e+00 0.0000000e+00]] (class=10)


In [ ]:
predict_row

array([1.   , 0.9  , 0.375])

## **Generate a SavedModel**

In [ ]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

## **Convert the SavedModel to TFLite**

In [ ]:
# convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converted.convert()

In [ ]:
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

## **Initialize the TFLite Interpreter to Try It Out**

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
# Test the TensorFlow Lite model on random input data.
input_shape = input_details[0]['shape']
inputs, outputs = [], []

# ambil data dari github untuk testing

## **Download the TFLite Model File**

In [ ]:
try:
    from google.colab import files
    files.download(tflite_model_file)
except:
    pass

# **Notes and Example**

## **Debugging**

The following list describes possible actions to debug model. To debug in ML requires us to sort through multiple possibilities at once. If an action sounds promising, experiment by modifying the code above.


*   Transforming data
*   Activation function
* **Hyperparameter values**
* Simpler model
* Change optimizer

Consider these actions and experiment where necessary



When debugging ML models, you should first attempt to diagnose the problem and apply the appropriate fix. For example, if you had changed your optimizer using `optimizer='sgd'`, then your model also converges faster. However, the problem was not with the optimizer but with the learning rate. Changing the optimizer only helps because `optimizer='sgd'` has a higher default learning rate than `optimizer='adam'`.

Alternatively, you could train the model for longer with the default learning rate. However, in real-world ML, models take long to train. You should keep your training cycles as short as possible. Therefore, increasing the learning rate is the correct fix.

These options demonstrate how debugging in ML is n-dimensional, and therefore you must use your understanding of model mechanics to narrow down your options. Because running experiments in ML is time consuming, requires careful setup, and can be subject to reproducibility issues, it's important to use your understanding of model mechanics to  narrow down options without having to experiment.

Lastly, according to development best practices, you should transform your feature data appropriately. This Colab did not transform the feature data because transformation is not required for convergence. However, you should always transform data appropriately. Here, you could normalize your feature data using z-score or scale the feature data to [0,1]. 

## **Solution: Reaching Convergence**

Of your loss isn't decreasing fast enough, from the guidance on [Learning Rate](https://developers.google.com/machine-learning/crash-course/reducing-loss/learning-rate), you know that you can increase the learning rate to train faster. Run the following code to increase the learning rate to 0.1. The the model reaches convergence quickly.

In [ ]:
model = None
model = keras.Sequential()
model.add(keras.layers.Dense(1, activation='linear', input_dim=1))
model.compile(optimizer=tf.optimizers.Adam(0.1), loss='mse')
trainHistory = model.fit(features, labels, epochs=5, batch_size=1, verbose=1)
# Plot loss curve
plt.plot(trainHistory.history['loss'])
plt.title('Loss Curves')

In [ ]:
#set random seed
tf.random.set_seed(42)

#create the model
model_12=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(4,activation="relu"),
    tf.keras.layers.Dense(4,activation="relu"),
    tf.keras.layers.Dense(10,activation="softmax")
])

#compile the model
model_12.compile(
   loss=tf.keras.losses.SparseCategoricalCrossentropy(),
   optimizer=tf.keras.optimizers.Adam(),
   metrics="accuracy")

#create a learning rate callback
lr_scheduler = tf.keras.callbacks
    .LearningRateScheduler(lambda epoch : 1e-3 *10**(epoch/20) )

#fit the model
fit_lr_history =model_12.fit(
   train_data_norm,
   train_labels,
   epochs=40,
   callbacks=[lr_scheduler],
   validation_data=(test_data_norm,test_labels))

IndentationError: ignored

### **Callback**

In [ ]:
# masih contoh
#plot the learniing rate curve
import numpy as np
import matplotlib.pyplot as plt
lrs =1e-3 *(10**(tf.range(40)/20))
plt.semilogx(lrs,fit_lr_history.history["loss"])
plt.xlabel("Learning Rate")
plt.ylabel("Loss")
plt.title("Finding the ideal Learning Rate")